In [1]:
%pip install -r requirements.txt -qU

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mlflow
import os
from getpass import getpass
import pandas as pd
from openai import OpenAI
from utils import get_random_files, get_image, get_json, _set_openai_api_key_for_demo

# Set MLflow tracking URI to cwd()
mlflow.set_tracking_uri(os.getcwd() + "/mlruns")
mlflow.set_experiment("evaluation")

# Get API key securely
if (not _set_openai_api_key_for_demo()) and (not os.getenv("OPENAI_API_KEY")):
    os.environ["OPENAI_API_KEY"] = getpass("Your OpenAI API Key: ")

2025/05/12 23:47:28 INFO mlflow.tracking.fluent: Experiment with name 'evaluation' does not exist. Creating a new experiment.


## Step 1 - Determine Accuracy

In [3]:
_files = get_random_files(n=5)
images = [get_image(file, encode_as_str=True) for file in _files]
jsons = [get_json(file) for file in _files]

mlflow.openai.autolog()
client = OpenAI()

In [4]:
system_prompt = """You are an expert at Optical Character Recognition (OCR). Extract the questions and answers from the image."""

def get_completion(inputs: str) -> str:
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": "what's in this image?" },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs}",
                        },
                    },
                ],
            }
        ],
    )

    return completion.choices[0].message.content

with mlflow.start_run() as run:
    predicted = get_completion(images[0])
    print(predicted)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


This image is a "Magazine Insertion Order" from Lorillard Media Services. It includes details such as:

- **To**: Essence magazine, attention Joyce Winston.
- **Date**: March 17, 1995.
- **Advertiser**: Lorillard.
- **Product**: Newport cigarettes.
- **Advertisement Requirements**: Complies with FTC requirements for warning statements and tar and nicotine lines in cigarette advertising.
- **Ad Details**: 
  - AD#: NPT-533-7
  - Caption: Fountain Couple
  - Space:  F5CB
  - Date: June 1995
- **Position Requested**: Far forward, opposite a full editorial.
- **Instructions**: Maintain separation from competitive ads and non-editorial content related to cigarettes.
- **Copy Instructions**: 5 color proofs attached.
- **Space Billing Instructions**: Ordered by Lorillard Media Services, invoices sent to Steve Molloy.

It includes several important instructions for making sure the advertisement is inserted correctly and requirements for the billing process.


In [5]:
jsons[0]

{'TO:': 'ESSENCE 1500 BROADWAY NEW YORK, NY 10036',
 'ATTN:': 'JOYCE WINSTON',
 'DATE:': 'JUNE 1995',
 'ADVERTISER:': 'LORILLARD',
 'PRODUCT:': 'NEWPORT',
 'AD#:': 'NPT- 533- 7',
 'CAPTION:': 'FOUNTAIN COUPLE',
 'SPACE:': 'P5CE',
 'PO': '#77219'}

In [6]:
predicted == jsons[0]

False

## Step 2: MLflow Evaluate

In [7]:
%pip install tiktoken aiohttp -qU

Note: you may need to restart the kernel to use updated packages.


In [8]:
correct_format = mlflow.metrics.genai.make_genai_metric(
    name="correct_format",
    definition=(
        """The answer is a list of dicts where keys are `question` and `answer`."""
    ),
    grading_prompt=(
        """If formatted correctly, return 1. Otherwise, return 0."""
    ),
    model="openai:/gpt-4o-mini",
    greater_is_better=True,
)

def batch_completion(df: pd.DataFrame) -> list[str]:
    return [get_completion(image) for image in df["inputs"]]

eval_result = mlflow.evaluate(
    model=batch_completion,
    data=pd.DataFrame({"inputs": images, "truth": jsons}),
    targets="truth",
    model_type="text",
    extra_metrics=[correct_format],
)

2025/05/12 23:47:47 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/05/12 23:47:47 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025/05/12 23:48:24 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/12 23:48:27 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundE

In [9]:
eval_result.tables['eval_results_table']

,inputs,truth,outputs,token_count,correct_format/v1/score,correct_format/v1/justification
0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABQODxIPDRQSEB...,"{'TO:': 'ESSENCE 1500 BROADWAY NEW YORK, NY 10...","This image is a ""Magazine Insertion Order"" fro...",184,0,The output does not match the required format ...
1,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABQODxIPDRQSEB...,"{'PARTIAL': 'X', 'TO:': 'S. P. ZOLOT', 'FROM:'...",This image is a progress report document title...,465,0,The output does not follow the required format...
2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABQODxIPDRQSEB...,"{'TO:': 'Mrs. K. A. Sparrow', 'FROM:': 'R. G. ...","The image is a progress report titled ""Newport...",190,0,The output does not match the required format ...
3,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABQODxIPDRQSEB...,"{'Date:': 'September 22, 1997', 'To:': 'Ron Mi...","The image is a fax cover sheet from ""Roper Sta...",107,0,The output does not match the required format ...
4,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABQODxIPDRQSEB...,"{'TO:': 'MRS. K. A. SPARROW', 'FROM:': 'R. G. ...","The image is a ""Competitive Product Introducti...",190,0,The output does not match the required format ...


In [10]:
print(eval_result.metrics)

{'correct_format/v1/mean': np.float64(0.0), 'correct_format/v1/variance': np.float64(0.0), 'correct_format/v1/p90': np.float64(0.0)}


## Step 3: Go to the UI 